In [1]:
import os,sys

# Import functions from functions.py
cur_dir = os.path.abspath('.')
if cur_dir not in sys.path:
    sys.path.append(cur_dir)  # Ensure current directory is in path
backend_dir = os.path.abspath('../backend')
if backend_dir not in sys.path:
    sys.path.append(backend_dir)  # Ensure backend directory is in path
# print(sys.path)

root_dir = os.path.abspath(f"..")
output_dir = os.path.join(root_dir, "output-data", "histogram")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
data_dir = os.path.join(root_dir, "test-data")

In [2]:
in_territorial_authority = os.path.join(data_dir, "statsnz-territorial-authority-2025-clipped-SHP/territorial-authority-2025-clipped.shp")
raster_types = [
    ("slope",os.path.join(data_dir, "lris-lenz-slope-GTiff/slope_2193.tif")),
    ("solar",os.path.join(data_dir, "lris-lenz-mean-annual-solar-radiation-GTiff/solar_2193.tif")),
    ("temperature",os.path.join(data_dir, "lris-lenz-mean-annual-temperature-GTiff/temperature_2193.tif"))
]

In [ ]:
import os, rasterio, numpy
import app.gis.tools as tools
import app.gis.consts as consts
import logging

from app.gis.functions import RPL_Select_analysis, RPL_ExtractByMask

# loop all districts
# clip boundary for single district
def clip_district_boundary(district_code):
    district_boundary_shp = os.path.join(output_dir, f"district_boundary.shp")
    RPL_Select_analysis(
        in_territorial_authority,
        district_boundary_shp,
        f"TA2025_V1_ == '{district_code}'"
    )
    return district_boundary_shp

def raster_histogram(raster_path):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        if src.nodata is not None:
            data = data[data != src.nodata]
        hist, edges = numpy.histogram(data, bins=10)
        # print(hist, edges)
        min_val = float(numpy.min(data))
        max_val = float(numpy.max(data))
    # Convert numpy arrays to lists of floats for JSON serialization
    return hist.astype(int).tolist(), [round(x, 2) for x in edges.astype(float).tolist()], min_val, max_val

def district_histograms(district_boundary_shp):
    raster_in_boundary = os.path.join(output_dir, f"raster_in_boundary.tif")
    rst = {}
    for kind, path in raster_types:
        RPL_ExtractByMask(path, district_boundary_shp, raster_in_boundary)
        hist, edges, min_val, max_val = raster_histogram(raster_in_boundary)
        rst[kind] = {
            "frequency": hist,
            "edges": edges,
            "min": min_val,
            "max": max_val
        }
    return rst

full = {}
for code, name in consts.districts:
    try:
        print(f"Processing district {code} - {name}")
        district_boundary_shp = clip_district_boundary(code)
        histograms = district_histograms(district_boundary_shp)
        full[code] = histograms
        # break
    except Exception as e:
        logging.error(f"Error processing district {code} - {name}: {e}")

# save full to a json file
import json
with open(os.path.join(output_dir, "district_histograms.json"), "w") as f:
    json.dump(full, f)

Processing district 063 - Ashburton District
Processing district 076 - Auckland
Processing district 076 - Auckland
Processing district 055 - Buller District
Processing district 055 - Buller District
Processing district 049 - Carterton District
Processing district 049 - Carterton District
Processing district 032 - Central Hawke's Bay District
Processing district 032 - Central Hawke's Bay District
Processing district 069 - Central Otago District
Processing district 069 - Central Otago District
Processing district 067 - Chatham Islands Territory
Processing district 067 - Chatham Islands Territory


ERROR:root:Error processing district 067 - Chatham Islands Territory: Input shapes do not overlap raster.


Processing district 060 - Christchurch City
Processing district 072 - Clutha District
Processing district 072 - Clutha District
Processing district 071 - Dunedin City
Processing district 071 - Dunedin City
Processing district 001 - Far North District
Processing district 001 - Far North District
Processing district 028 - Gisborne District
Processing district 028 - Gisborne District
Processing district 074 - Gore District
Processing district 074 - Gore District
Processing district 056 - Grey District
Processing district 056 - Grey District
Processing district 016 - Hamilton City
Processing district 016 - Hamilton City
Processing district 030 - Hastings District
Processing district 030 - Hastings District
Processing district 012 - Hauraki District
Processing district 012 - Hauraki District
Processing district 042 - Horowhenua District
Processing district 042 - Horowhenua District
Processing district 058 - Hurunui District
Processing district 058 - Hurunui District
Processing district 075 